In [6]:
import string
import warnings
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

warnings.filterwarnings("ignore")
data = {
    "hello": "Hi there! How can I assist you today?",
    "how are you": "I'm just a bot, but I'm doing great! How about you?",
    "what is your name": "I'm an AI chatbot created to help you.",
    "favorite travel destination": "Hill stations or beaches?",
    "bye": "See you! Have a great day!",
    "thank you": "You're welcome! Happy to help."

}

def clean_text(text):
    return text.lower().translate(str.maketrans('', '', string.punctuation))

def train_model():
    global model, vectorizer
    questions = list(data.keys())
    responses = list(data.values())

    vectorizer = CountVectorizer()
    X_train = vectorizer.fit_transform(questions)

    model = Pipeline([
        ('tfidf', TfidfTransformer()),
        ('clf', MultinomialNB())
    ])

    model.fit(X_train, responses)

def chatbot_response(user_input):
    user_input = clean_text(user_input)

    if user_input.startswith("add:"):
        try:
            parts = user_input.split(":", 2)
            if len(parts) == 3:
                _, new_question, new_response = parts
                new_question = clean_text(new_question.strip())
                new_response = new_response.strip()

                if new_question and new_response:
                    data[new_question] = new_response
                    train_model()
                    return f"Added new question: '{new_question}' → '{new_response}'"
                else:
                    return "Invalid format. Use: add:question:response"
            else:
                return "Invalid format. Use: add:question:response"
        except Exception as e:
            return f"Error: {e}"

    try:
        X_test = vectorizer.transform([user_input])
        return model.predict(X_test)[0]
    except:
        return "I'm sorry, I didn't understand that."

train_model()

print("Chatbot: Hello! Type 'bye' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "bye":
        print("Chatbot:", data["bye"])
        break
    print("Chatbot:", chatbot_response(user_input))


Chatbot: Hello! Type 'bye' to exit.
You: hello
Chatbot: Hi there! How can I assist you today?
You: how  are you
Chatbot: I'm just a bot, but I'm doing great! How about you?
You: what is your name
Chatbot: I'm an AI chatbot created to help you.
You: bye
Chatbot: See you! Have a great day!
